# Module 8 - Logistic Regression

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
location = "datasets/titanic.xls"

df = pd.read_excel(location)
df.head()

#### Explore through visualized data

In [ ]:
#bar chart of survival status count
sns.countplot(x='survived', data=df)

In [ ]:
#survival status by sex
sns.barplot('sex', 'survived', data=df)

In [ ]:
#survival rate by passenger class
sns.barplot('pclass', 'survived', data=df)

In [ ]:
#plot age by survival status
plt.figure(figsize=(10,6))

ax = sns.kdeplot(df['age'][df['survived'] == 1], #passengers that survived
                 color="darkturquoise",
                 shade=True)

sns.kdeplot(df['age'][df['survived'] == 0], #passengers that did not survive
            color="lightcoral",
            shade=True)

plt.legend (['Survived', 'Died'])
plt.title("Density Plot of Age for Survived vs Deceased Population")

ax.set(xlabel='Age')
#plt.show()

#### Handle missing values

In [ ]:
#find columns that have missing values
df.isnull().sum()

Let's clean up 'age' and 'embarked'

In [ ]:
#rows where the age is missing
missing_age = df.loc[df['age'].isnull()]
missing_age.head()

In [ ]:
#get index numbers of missing rows - we'll use this later
mals = list(missing_age.index)

In [ ]:
#table of avg age of passenger by survival status, sex, and passenger class
df.groupby(['survived', 'sex', 'pclass'])['age'].mean()

In [ ]:
#fill missing values for age based on survival status, sex, and passenger class
df['age'].fillna(df.groupby(['survived', 'sex', 'pclass'])['age'].transform('mean'), inplace=True)

In [ ]:
#verify filled missing values 
df.iloc[mals].head()

In [ ]:
#verify there are no more missing age values
df.isnull().sum()

In [ ]:
#missing values for 'embarked'
embark = df.loc[df['embarked'].isnull()]
embark

In [ ]:
#save index for missing values to verify later
embarkls = list(embark.index)

In [ ]:
#only 2 missing values so we'll fill with most common embarkation point
df['embarked'].value_counts()

In [ ]:
#fill missing values
df['embarked'].fillna('S', inplace=True)

In [ ]:
#check that they're filled
df.iloc[embarkls]

In [ ]:
df.isnull().sum()

Get rid of columns that we don't want to use in the model

In [ ]:
modeldf = df.drop(['name','ticket','fare', 'cabin', 'boat', 'body', 'home.dest'], axis=1)

In [ ]:
#columns left in our dataframe
modeldf.columns

Create dummy variables for categorical values

In [ ]:
#dummy variables for passenger class embarkation port
#get_dummies will auto-drop columns that dummies were created from
modeldf = pd.get_dummies(data=modeldf, columns=['pclass','embarked'])
modeldf.head()

In [ ]:
#change sex values to binary
#female=0, male=1
modeldf['sex'] = modeldf['sex'].map({'female':0, 'male':1})
modeldf.head()

In [ ]:
#create new column based on number of family members
#drop sibsp and parch columns
modeldf['family_num'] = modeldf['sibsp'] + modeldf['parch']
modeldf.drop(['sibsp', 'parch'], axis=1, inplace=True)
modeldf.head()

In [ ]:
modeldf['TravelAlone']=np.where((modeldf['family_num'] > 0), 0, 1)
modeldf.head()

## Logistic Regression

#### Split data into train and test

In [ ]:
#extract target variable
#make copy of 'survived' column
y = modeldf['survived']

In [ ]:
#copy of modeldf without 'survived' column
X = modeldf.drop(['survived'], axis=1)

In [ ]:
#80% for training data, 20% for test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

In [ ]:
#build logistic regression model
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

In [ ]:
#accuracy score of model using training data
LogReg.score(X_train, y_train)

In [ ]:
#generate prediction values
y_pred = LogReg.predict(X_test)

In [ ]:
#Confusion matrix shows which values model predicted correctly vs incorrectly

cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Not Survival', 'Predicted Survival'],
    index=['True Not Survival', 'True Survival']
)

cm

In [ ]:
#accuracy score of model on test data
LogReg.score(X_test, y_test)

In [ ]:
#from precision column, model is better at predicting passengers that do not survive
print(classification_report(y_test, y_pred))

Sources:

https://nbviewer.jupyter.org/github/BigDataGal/Data-Mania-Demos/blob/master/Logistic%20Regression%20Demo.ipynb

https://mashimo.wordpress.com/2018/03/31/logistic-regression-using-sklearn/

https://www.kaggle.com/mnassrib/titanic-logistic-regression-with-python/notebook

https://datascienceplus.com/would-you-survive-the-titanic-getting-started-in-python/

https://towardsdatascience.com/predicting-the-survival-of-titanic-passengers-30870ccc7e8